# 1-D Facet spaces and hybrid methods (Unit 2.7)
# Won't work (no H(div))

Mixed methods for second order problems lead to saddle point problems, and indefinite matrices. By hybridization one obtains a positive definite system again. It's structure is similar to the non-conforming $\mathcal{P}^1$ method, but hybridization works for any order. See text-book by Brezzi and Fortin.

One skips the normal-continuity of the $H(div)$ variable, and reinforces it by a Lagrange parameter. This leads to the following discrete system:

Find $\sigma, u, \hat{u} \in \Sigma_h \times V_h \times F_h$:

$$\int \lambda^{-1} \sigma \tau + \sum_T \int_T \text{ div } \tau u + \sum_F \int_F [\tau n] \hat{u} = 0 \ \ \forall \tau \in \Sigma_h$$

$$\int \text{div } \sigma v = \int fv \ \ \forall v \in V_h$$

$$\int [\sigma n] \hat{v} = \int_{\Gamma_n} g \hat{v} \ \ \forall \hat{v} \in F_h $$ 

where $\Sigma_h$ is a discontinuous $H(div)$ finite element space, $V_h$ a sub-space of $L_2$, and $F_h$ consists of polynomials on every edge.

### Notes: 
I think $F_h$ stands for Facet

What is the problem we're trying to solve here?  I think it's exactly the diffusion problem in Mixed. The main difference is that now we have $\hat{u}$ (edge polynomials) and $\sigma$ can be discontinous.  Open the Mixed notebook and compare!

We can use the 'discontinuous' keyword for a finite element space to specify whether a space is discontinuous.  Read on...

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))


same example as in 'mixed':

In [2]:
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([y*(1-y) if bc=="left" else 0 for bc in mesh.GetBoundaries()])
lam = 10

define spaces:

- The discontinuous flag generates an element-wise $H(Div)$-space
- FacetFESpace lives only on facets (i.e. faces in 3D, edges in 2D, points in 1D)

Boundary conditions are now posed for the facet-space

In [3]:
order = 3
V = HDiv(mesh, order=order, discontinuous = True)
Q = L2(mesh, order=order-1)
F = FacetFESpace(mesh, order=order, dirichlet="bottom")
X = FESpace([V,Q,F])


Assemble forms. The jump-term is rewritten as

$$\sum_F \int_F [\sigma_n] v = \sum_T \int_{\partial T} \sigma_n v$$


Ok, so sum of integrals of jumps on facets equals sum of integrals over element boundaries... 

Is the jump term $[\sigma_n]=\sigma_{n_{left}}−\sigma_{n_{right}}$?

In [4]:
sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

a = BilinearForm(X)
a += SymbolicBFI(1/lam * sigma*tau + div(sigma)*v + div(tau)*u)
n = specialcf.normal(mesh.dim)
a += SymbolicBFI(sigma*n*vhat+tau*n*uhat, element_boundary=True)
a.Assemble()

# Compare with mixed!
# am = BilinearForm(fesm)
# am += SymbolicBFI(1/lam * sigma * tau + div(sigma)*v + div(tau)* u)
# am.Assemble()

f = LinearForm(X)
f += SymbolicLFI(-source*v)
f += SymbolicLFI(g*vhat, BND)
f.Assemble()

# Compare with mixed!
# fm = LinearForm(fesm)
# fm += SymbolicLFI(-source*v)
# normal = specialcf.normal(mesh.dim)
# fm += SymbolicLFI(ud * (tau.Trace()*normal), BND)
# fm.Assemble()



RuntimeError: cannot evaluate facet-fe inside element, add trans simdin Assemble LinearForm


Solve system. Leave everything to the sparse direct solver.

In [ ]:
gfu = GridFunction(X)
inv = a.mat.Inverse(freedofs=X.FreeDofs())
gfu.vec.data = inv * f.vec


In [ ]:
Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "u")